In [1]:
import matplotlib.pyplot as plt
import numpy as np
import IPython
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow_io as tfio
import functools
from pedalboard import load_plugin
from ReverberatorEstimator import layers, loss
import warnings
warnings.filterwarnings('ignore')
import time
import os
import librosa.display

2021-12-06 09:00:41.297022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 09:00:41.297083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sample_rate = 48000
num_params = 28

In [3]:
target_audio = tfio.audio.AudioIOTensor("Dataset/Wet/Snap.wav")
target_audio = target_audio.to_tensor()
target_audio = tf.slice(target_audio, begin=[0,0], size=[-1,1])
target_audio = tf.cast(target_audio, tf.float32) / 32768.0
target_audio = tf.squeeze(target_audio)
target_audio = tf.reshape(target_audio,(1, 96000))
input_audio = tfio.audio.AudioIOTensor("Dataset/Dry/Snap.wav")
input_audio = input_audio.to_tensor()
input_audio = tf.slice(input_audio, begin=[0,0], size=[-1,1])
input_audio = tf.cast(input_audio, tf.float32) / 32768.0
input_audio = tf.squeeze(input_audio)
input_audio = tf.reshape(input_audio,(1, 96000))

2021-12-06 09:00:56.622563: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2021-12-06 09:00:57.230939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 09:00:57.231178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-06 09:00:57.231366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-06 09:00:57.231536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dl

In [4]:
dataset_path = os.path.abspath("./Dataset")

dry_files = []

for f in range(8):
    audio = tfio.audio.AudioIOTensor(dataset_path + "/Dry/Snap.wav")
    audio = audio.to_tensor()
    audio = tf.slice(audio, begin=[0,0], size=[-1,1])
    audio = tf.cast(audio, tf.float32) / 32768.0
    audio = tf.squeeze(audio)
    audio = tf.reshape(audio,(96000))
    dry_files.append(audio)
    
x_train = tf.stack(dry_files)

wet_files = []

for f in range(8):
    audio = tfio.audio.AudioIOTensor(dataset_path + "/Wet/Snap.wav")
    audio = audio.to_tensor()
    audio = tf.slice(audio, begin=[0,0], size=[-1,1])
    audio = tf.cast(audio, tf.float32) / 32768.0
    audio = tf.squeeze(audio)
    audio = tf.reshape(audio,(96000))
    wet_files.append(audio)
    
y_train = tf.stack(wet_files)

In [5]:
logmelgram = layers.LogMelgramLayer(1024, 256, 128, sample_rate, 0.0, sample_rate//2, 1e-6)
audio_time = tfkl.Input(shape=(96000,), name="audio_time")

x = logmelgram(audio_time)
x = tfkl.BatchNormalization(name="input_norm")(x)
encoder_model = tfk.applications.MobileNetV2(input_shape=(x.shape[1], x.shape[2], x.shape[3]), alpha=1.0,
                                            include_top=True, weights=None, input_tensor=None, pooling=None,
                                            classes=np.sum(num_params).item(), classifier_activation="sigmoid")

hidden = encoder_model(x)

parameter_model = tfk.models.Model(audio_time, hidden, name="parameter_model")

parameters = parameter_model(audio_time)

vstlayer = layers.VSTProcessor("../Reverberator.vst3", sample_rate)
output = vstlayer([audio_time, parameters])

model = tfk.models.Model(audio_time, output, name="full_model")

spectral_loss = loss.multiScaleSpectralLoss(sr=sample_rate)

optimizer = tfk.optimizers.Adam(learning_rate=0.001)

checkpoint_dir = './training_checkpoints'

model.compile(optimizer=optimizer, loss=spectral_loss, metrics=['mae'], run_eagerly=True)

No protocol specified
No protocol specified
No protocol specified
No protocol specified


In [6]:
parameter_model.summary()
model.summary()

Model: "parameter_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
audio_time (InputLayer)      [(None, 96000)]           0         
_________________________________________________________________
log_melgram_layer (LogMelgra (None, 372, 128, 1)       0         
_________________________________________________________________
input_norm (BatchNormalizati (None, 372, 128, 1)       4         
_________________________________________________________________
mobilenetv2_1.00_372 (Functi (None, 28)                2293276   
Total params: 2,293,280
Trainable params: 2,259,166
Non-trainable params: 34,114
_________________________________________________________________
Model: "full_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
audio_time (Inpu

In [9]:
# Restore from latest checkpoint
model.load_weights(checkpoint_dir)

2021-12-06 09:03:09.453504: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ./training_checkpoints: Failed precondition: training_checkpoints; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [10]:
audio_pre = (model.call(input_audio)).numpy()[0]
old_params = parameter_model(input_audio).numpy()[0]
print(old_params)

[0.95932823 0.96229666 0.9793441  0.98757935 0.98352724 0.97451365
 0.9774519  0.9812577  0.97978985 0.98163354 0.9826826  0.9884155
 0.00200912 0.9861465  0.97938055 0.99001485 0.14924392 0.24214837
 0.47357398 0.60786474 0.58504057 0.95497465 0.15346733 0.9481719
 0.8466417  0.9603492  0.24379805 0.98521864]


In [11]:
model_cp = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                  factor=0.2,
                                  patience=10,
                                  cooldown=0,
                                  verbose=1,
                                  mode='min',
                                  min_lr=0.0000016)

In [ ]:
start_time = time.time()
history = model.fit(x_train, y_train, verbose=1, epochs=1000,
         callbacks=[model_cp,reduce_lr])
print("Training took %d seconds" % (time.time() - start_time))

2021-12-06 09:03:24.153153: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00001: loss improved from inf to 0.68486, saving model to ./training_checkpoints


2021-12-06 09:03:39.592331: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./training_checkpoints/assets
Epoch 2/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0297

Epoch 00002: loss did not improve from 0.68486
Epoch 3/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0297

Epoch 00003: loss did not improve from 0.68486
Epoch 4/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0297

Epoch 00004: loss did not improve from 0.68486
Epoch 5/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00005: loss did not improve from 0.68486
Epoch 6/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00006: loss did not improve from 0.68486
Epoch 7/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00007: loss did not improve from 0.68486
Epoch 8/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Ep

1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00059: loss did not improve from 0.68485
Epoch 60/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00060: loss did not improve from 0.68485
Epoch 61/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00061: loss did not improve from 0.68485
Epoch 62/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00062: loss did not improve from 0.68485
Epoch 63/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00063: loss did not improve from 0.68485
Epoch 64/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00064: loss did not improve from 0.68485
Epoch 65/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00065: loss did not improve from 0.68485
Epoch 66/1000
1/1 [========

1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00116: loss did not improve from 0.68484
Epoch 117/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00117: loss did not improve from 0.68484
Epoch 118/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00118: loss did not improve from 0.68484
Epoch 119/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00119: loss did not improve from 0.68484
Epoch 120/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00120: loss did not improve from 0.68484
Epoch 121/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00121: loss did not improve from 0.68484
Epoch 122/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00122: loss did not improve from 0.68484
Epoch 123/1000
1/1 [=

1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00174: loss did not improve from 0.68484
Epoch 175/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00175: loss did not improve from 0.68484
Epoch 176/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00176: loss did not improve from 0.68484
Epoch 177/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00177: loss did not improve from 0.68484
Epoch 178/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00178: loss did not improve from 0.68484
Epoch 179/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00179: loss did not improve from 0.68484
Epoch 180/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00180: loss did not improve from 0.68484
Epoch 181/1000
1/1 [=

1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0298

Epoch 00232: loss did not improve from 0.68484
Epoch 233/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00233: loss did not improve from 0.68484
Epoch 234/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00234: loss did not improve from 0.68484
Epoch 235/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00235: loss did not improve from 0.68484
Epoch 236/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00236: loss did not improve from 0.68484
Epoch 237/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00237: loss did not improve from 0.68484
Epoch 238/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6850 - mae: 0.0299

Epoch 00238: loss did not improve from 0.68484
Epoch 239/1000
1/1 [=

1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00290: loss did not improve from 0.68484
Epoch 291/1000
1/1 [==============================] - 5s 5s/step - loss: 0.6849 - mae: 0.0299

Epoch 00291: loss did not improve from 0.68484
Epoch 292/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00292: loss did not improve from 0.68484
Epoch 293/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0299

Epoch 00293: loss did not improve from 0.68484
Epoch 294/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00294: loss did not improve from 0.68484
Epoch 295/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00295: loss did not improve from 0.68484
Epoch 296/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0298

Epoch 00296: loss did not improve from 0.68484
Epoch 297/1000
1/1 [=

1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00348: loss did not improve from 0.68484
Epoch 349/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00349: loss did not improve from 0.68484
Epoch 350/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00350: loss did not improve from 0.68484
Epoch 351/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00351: loss did not improve from 0.68484
Epoch 352/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00352: loss did not improve from 0.68484
Epoch 353/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00353: loss did not improve from 0.68484
Epoch 354/1000
1/1 [==============================] - 4s 4s/step - loss: 0.6849 - mae: 0.0297

Epoch 00354: loss did not improve from 0.68484
Epoch 355/1000
1/1 [=

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
output_audio = model(input_audio)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7,10))
ax[0].plot(audio_pre)
D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_pre)), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1])

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))
ax[0,0].plot(output_audio.numpy()[0])
D = librosa.amplitude_to_db(np.abs(librosa.stft(output_audio.numpy()[0])), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1,0])
ax[0,1].plot(target_audio.numpy()[0])
D = librosa.amplitude_to_db(np.abs(librosa.stft(target_audio.numpy()[0])), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1,1])

In [ ]:
IPython.display.Audio(output_audio, rate=sample_rate, autoplay=True)

In [ ]:
IPython.display.Audio(audio_pre, rate=sample_rate)

In [ ]:
IPython.display.Audio(target_audio, rate=sample_rate)

In [ ]:
# Dump values to .csv files
np.savetxt("output_audio.csv", output_audio.numpy()[0], delimiter=",")
np.savetxt("target_audio.csv", target_audio.numpy()[0], delimiter=",")

In [ ]:
parameters = parameter_model(input_audio)

In [ ]:
params = parameters.numpy()[0]
filter_c = 1
for i in range(num_params):
    if i < 4:
        print("b_%i = %f" % (i, params[i]))
    elif i < 8:
        print("c_%i = %f" % (i-4, params[i]))
    else:
        j = (i-8) % 5
        
        if j is 0:
            print("\nFilter %i:" % filter_c)
            filter_c = filter_c + 1
            print("c_hp = %f" % params[i])
        elif j is 1:
            print("c_bp = %f" % params[i])
        elif j is 2:
            print("c_lp = %f" % params[i])
        elif j is 3:
            print("g = %f" % params[i])
        elif j is 4:
            print("R = %f" % params[i])

In [ ]:
param_diff = params - old_params
print(param_diff)
plt.stem(param_diff)
plt.ylim([-1, 1])